# <center> Parallel Sorting: MapReduce TeraSort </center>

O’Malley, Owen. "Terabyte sort on apache hadoop." Yahoo, available online at: http://sortbenchmark.org/Yahoo-Hadoop.pdf, (May) (2008): 1-3.

- Won the general purpose terabyte sort benchmark in 2008 and 2009
- 2008
    - 910 nodes
    - 4 dual-core Xeons @ 2.0ghz, 4 SATA disks, 8G RAM, 1 gigabit ethernet
    - 40 nodes per rack
    - **3.48 minutes**
- 2009
    - ~3800 nodes
    - 2 quad-core Xesons @ 2.5ghz, 4 SATA disks, 8G RAM, 1 gigabit ethernet
    - 40 nodes per rack
    - ** 62 seconds ** (1TB sort - this benchmark was retired after 2008)
    - ** 975 minutes ** (1PB sort)

Terasort includes 3 MapReduce steps:
    
- Teragen: generates data
- Terasort: sorts data
- Teravalidate: validates the output is sorted

In [3]:
# generate 100GB of random data 
!ssh dsciu001 hdfs dfs -rm -r tmp/teragenout
!yarn jar \
    /usr/hdp/current/hadoop-mapreduce-client/hadoop-mapreduce-examples-*.jar \
    teragen -Dmapred.map.tasks=700 100000000000 tmp/teragenout

17/03/07 21:47:52 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 360 minutes, Emptier interval = 0 minutes.
Moved: 'hdfs://dsci/user/lngo/tmp/teragenout' to trash at: hdfs://dsci/user/lngo/.Trash/Current
17/03/07 21:47:55 INFO impl.TimelineClientImpl: Timeline service address: http://dscim003.palmetto.clemson.edu:8188/ws/v1/timeline/
17/03/07 21:47:56 INFO hdfs.DFSClient: Created HDFS_DELEGATION_TOKEN token 6475 for lngo on ha-hdfs:dsci
17/03/07 21:47:56 INFO security.TokenCache: Got dt for hdfs://dsci; Kind: HDFS_DELEGATION_TOKEN, Service: ha-hdfs:dsci, Ident: (HDFS_DELEGATION_TOKEN token 6475 for lngo)
17/03/07 21:47:56 INFO terasort.TeraSort: Generating 100000000000 using 700
17/03/07 21:47:56 INFO mapreduce.JobSubmitter: number of splits:700
17/03/07 21:47:56 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1488826571229_0023
17/03/07 21:47:56 INFO mapreduce.JobSubmitter: Kind: HDFS_DELEGATION_TOKEN, Service: ha-hdfs:dsci, Ident: (HDFS_DELE

In [4]:
# sort 100GB of data
!yarn jar \
    /usr/hdp/current/hadoop-mapreduce-client/hadoop-mapreduce-examples-*.\
    terasort -Dmapred.reduce.tasks=700 tmp/teragenout tmp/terasortout

17/03/07 22:02:00 INFO terasort.TeraSort: starting
17/03/07 22:02:01 INFO hdfs.DFSClient: Created HDFS_DELEGATION_TOKEN token 6476 for lngo on ha-hdfs:dsci
17/03/07 22:02:01 INFO security.TokenCache: Got dt for hdfs://dsci; Kind: HDFS_DELEGATION_TOKEN, Service: ha-hdfs:dsci, Ident: (HDFS_DELEGATION_TOKEN token 6476 for lngo)
17/03/07 22:02:03 INFO input.FileInputFormat: Total input paths to process : 700
Spent 1979ms computing base-splits.
Spent 200ms computing TeraScheduler splits.
Computing input splits took 2180ms
Sampling 10 splits of 74900
Making 700 from 100000 sampled records
Computing parititions took 680ms
Spent 2862ms computing partitions.
17/03/07 22:02:04 INFO impl.TimelineClientImpl: Timeline service address: http://dscim003.palmetto.clemson.edu:8188/ws/v1/timeline/
17/03/07 22:02:05 INFO mapreduce.JobSubmitter: number of splits:74900
17/03/07 22:02:05 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1488826571229_0024
17/03/07 22:02:05 INFO mapreduce.JobSubmitt

Terasort in a nutshell: Bucketsort based on Keys

- Map processes data and emits items to be sorted as Keys and some random value as filler
- Assumption: Ranges of values of keys are known
- Customized Partitioner: balance and distribute keys to various reducers such that *keys(i-1) < keys(i) < keys (i+1)*
- Reducers sort their own set of keys
- Profit!!!

Partitioner design is based on trie tree

Trie tree advantages (https://en.wikipedia.org/wiki/Trie)

- Data look up is faster in the worst case
- No collisions of different key
- No hash function is needed
- **Alphabetical ordering of entries by key**

Trie tree disadvantages (https://en.wikipedia.org/wiki/Trie)

- Slower than hash table in looking up data (we are not looking up data)
- Floating point numbers can lead to chains and prefixes that are not meaningful (we don't care because we are not analyzing data)
- Some trie might require more space than a hash table (What is Hadoop for? big data)